### Capstone Idea

[GitHub](https://github.com/biborsz/Capstone)

**Problem Statement:** 

Fedbizopps.gov used to be a website where small businesses could search for federal contract opportunities. While the collection of soliciations and award notifications was searchable based on key terms, finding opportunities of interest based on similarity, rather than key term search, as far as I know, was not available. As a result, looking for contract opportunities took a considerable time, which could potentially put a strain on lightly staffed small businesses. The successor of Fedbizopps.gov, Beta.SAM.gov, although not yet fully operational, provides a wide array of filtering options besides the search by key terms and the browsing function. Those new to the system would benefit, however, from an application that recommends opportunities of interest based on topic and/or wording similarity. 

The purpose of this project is two-fold:
1. Stretching the limits of natural language processing, build an application that, based on user up- or downvote would recommend contract notifications of interest.
2. Observe how well a content based recommender system is capable of finding not only similar but also relevant notifications. 


-> it would give a list of potentially useful search terms, with an option to click on a select few

-> based on the search terms selected - it will further tune the recommender system

-> businesses have an option of rating a recommendation up or down

- based on earlier search terms - one would have access to a longer list of opportunites - that would make browsing possible (right now - it looks to me - that is out of the question)

- companies could search for other relevant information - for example: which companies are applying for similar grants in their business and geographic area
   
   
[API documentation](https://open.gsa.gov/api/get-opportunities-public-api/#user-account-api-key-creation)

[Beta.SAM.gov](https://beta.sam.gov/)

**Methodology:**
   - content based recommender system
   - text vectorizer: *bag-of-words*, *one-hot-encoder* 
     - bag of words: extracts words from the corpus as features
     - one-hot-encoder: gives a value of one if it finds a feature within a row/ text (0 to all other features)
   - classifies/ recommends based on *cosine similarities*

**Sources:**

https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d

https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html

http://recommender-systems.org/content-based-filtering/

https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

**Ideas for the future:**
 - find database of registered businesses (business activity/ line of business)
 - crossreference awardees in prior grants to provide a list of competitors

In [1]:
# imports
import pandas as pd
import numpy as np
import requests
import time
import datetime

In [2]:
# set display options 
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

In [ ]:
# rewrite code - to read in combined file - 
# separete out dates 
# split string '-'
# turn them into datetime object
# check for oldest date
# turn back into string
# make that postedTo date
# have for loop pull another page
# transform json object
# concat it to existing dataframe
# save it to csv
# I am at 03/02/2020

In [3]:
# define function to return minimum postedDate as string
def get_min_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    min_date = pd.to_datetime(df['postedDate']).min()
    return min_date.strftime('%m/%d/%Y')

In [4]:
# define function to return maximum postedDate as string
def get_max_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    min_date = pd.to_datetime(df['postedDate']).max()
    return min_date.strftime('%m/%d/%Y')

In [5]:
# define function to return today's date as string
def today():
    now = datetime.datetime.now()
    return now.strftime('%m/%d/%Y')

In [6]:
today()

'06/02/2020'

In [4]:
get_max_post_date('./data/combined.csv')

'05/14/2020'

In [7]:
# pull contract information from api.sam.gov

# set base url
url = 'https://api.sam.gov/prod/opportunities/v1/search'

# create empty list to store results
result = []
# initialize counter
count = 0
# downloaded contract opportunities - file name
# file = './data/combined.csv'

# set postedTo date to today's date by calling today() function
postedTo = today()

# set posted from date
postedFrom = get_max_post_date('./data/combined.csv')

# for loop to pull contracts
for i in range(10):
    count += 1
    
    # do a get request
    req = requests.get(url,
                      params={
                          'api_key': '',
                          'postedFrom': postedFrom,
                          'postedTo': postedTo,
                          'limit': 1000,
                          'offset': (count-1) * 1000
                      })
    
    # add response to result list
    result.append(req)
    
    now = datetime.datetime.now()
    print('Time:', now.strftime("%Y-%m-%d %H:%M:%S"))
    time.sleep(5)
    
    
    
# source for datetime - https://www.w3resource.com/python-exercises/python-basic-exercise-3.php

Time: 2020-06-02 21:38:44
Time: 2020-06-02 21:39:11
Time: 2020-06-02 21:39:35
Time: 2020-06-02 21:40:01
Time: 2020-06-02 21:40:15
Time: 2020-06-02 21:40:21
Time: 2020-06-02 21:40:30
Time: 2020-06-02 21:40:36
Time: 2020-06-02 21:40:44
Time: 2020-06-02 21:40:50


In [8]:
# unpack list of json objects from response data
ops = []
for item in result:
    print(item.headers)
    ops.append(item.json())

{'Age': '20', 'Content-Type': 'application/hal+json', 'Date': 'Wed, 03 Jun 2020 01:38:44 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.54.200, 10.177.54.200', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Set-Cookie': 'citrix_ns_id=f+6nyi/93TsbB4ymY/y/Sxn+npQ0000; Domain=.sam.gov; Path=/; Secure; HttpOnly', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked'}
{'Age': '20', 'Content-Type': 'application/hal+json', 'Date': 'Wed, 03 Jun 2020 01:39:10 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.54.200

In [9]:
# parse json objects
ls_data = []
for i in range(len(ops)):
    print(ops[i].keys())
    df = pd.DataFrame(ops[i]['opportunitiesData'])
    ls_data.append(df)
    data = pd.concat(ls_data)

dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])


In [10]:
data.shape

(4287, 27)

In [11]:
data.tail(10)

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,archiveDate,typeOfSetAsideDescription,typeOfSetAside,responseDeadLine,naicsCode,classificationCode,active,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
277,0354455d737643bc98f6a553922d64a3,Triton UAS MOB ECP Development,CJA-15_24847_N00019-15-G-0026_N00019-20-F-0865,DEPT OF DEFENSE,DEPT OF THE NAVY,NAVAL AIR SYSTEMS COMMAND,2020-05-29,Justification,Justification,auto30,2020-06-29,None,None,None,None,AC15,Yes,"{'date': '2020-05-15', 'number': 'N00019-15-G-0026'}","[{'fax': '', 'type': 'primary', 'email': 'jessica.l.guy@navy.mil', 'phone': '', 'title': None, 'fullName': 'Jessica L. Guy'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=0354455d737643bc98f6a553922d64a3,OFFICE,"{'zipcode': '20670-5000', 'city': 'PATUXENT RIVER', 'countryCode': 'USA', 'state': 'MD'}",None,None,https://beta.sam.gov/opp/0354455d737643bc98f6a553922d64a3/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=0354455d737643bc98f6a553922d64a3&limit=1'}]",[https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/6c67a84e117d4f038bdbd16e5101ea98/download?api_key=null&token=]
278,034d00eff98c4da6b553072cf0ba3d37,"53--BOLT,MACHINE",SPE5E820U0029,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA TROOP SUPPORT,2020-05-29,Award Notice,Award Notice,auto15,2020-06-13,None,None,None,332722,53,Yes,"{'date': '2020-05-29', 'number': 'SPE5E420D60CC', 'amount': '250000.00', 'awardee': {'name': 'Alamo Aircraft, Ltd. 2538 SW 36th St San Antonio 78237-4032 US', 'location': {}, 'duns': '027005180', 'cageCode': '55345'}}","[{'fax': None, 'type': 'primary', 'email': 'Lisa.Diplacido@dla.mil', 'phone': None, 'title': None, 'fullName': 'LISA DIPLACIDODSN444-0831'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=034d00eff98c4da6b553072cf0ba3d37,OFFICE,"{'zipcode': '19111', 'city': 'PHILADELPHIA', 'countryCode': 'USA', 'state': 'PA'}",{},None,https://beta.sam.gov/opp/034d00eff98c4da6b553072cf0ba3d37/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=034d00eff98c4da6b553072cf0ba3d37&limit=1'}]",None
279,0334dc835194483fa1888f4eba35ab9f,Replace Roofing Hospital & Education,15B41720B00000001,"JUSTICE, DEPARTMENT OF",FEDERAL PRISON SYSTEM / BUREAU OF PRISONS,FAO,2020-05-29,Presolicitation,Presolicitation,autocustom,2020-12-30,Total Small Business Set-Aside (FAR 19.5),SBA,2020-07-22T14:00:00-05:00,238160,Z2FF,Yes,None,"[{'fax': '9723524545', 'type': 'primary', 'email': 'mrmorgan@bop.gov', 'phone': '9723524504', 'title': None, 'fullName': 'Michelle R Morgan'}, {'fax': '', 'type': 'secondary', 'email': 'jkercheval@bop.gov', 'phone': '9723524520', 'title': None, 'fullName': 'Jeffrey P. Kercheval'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=0334dc835194483fa1888f4eba35ab9f,OFFICE,"{'zipcode': '75051', 'city': 'GRAND PRAIRIE', 'countryCode': 'USA', 'state': 'TX'}","{'city': {'code': '53920', 'name': 'Milan'}, 'state': {'code': 'MI', 'name': 'Michigan'}, 'zip': '48160', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",None,https://beta.sam.gov/opp/0334dc835194483fa1888f4eba35ab9f/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=0334dc835194483fa1888f4eba35ab9f&limit=1'}]",None
280,03081065fada472d98a28c8d0f89de5b,HVAC Horizontal,F1C0RH0125AW02,DEPT OF DEFENSE,DEPT OF THE AIR FORCE,FA5004 354 CONS PK,2020-05-29,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2020-06-20,Total Small Business Set-Aside (FAR 19.5),SBA,2020-06-05T14:00:00-08:00,33599,5975,Yes,None,"[{'fax': '', 'type': 'primary', 'email': 'jarred.julian@us.af.mil', 'phone': '9073773326', 'title': None, 'fullName': 'Jarred W. Julian'}, {'fax': '', 'type': 'secondary', 'email': 'brandon.tracer@us.af.mil', 'phone': '9073775309', 'title': None, 'fullName'

In [12]:
data.duplicated('noticeId').sum()

0

In [15]:
# data.drop_duplicates('noticeId', inplace=True)

In [13]:
data.to_csv('./data/6_02_pull.csv', index=False)

In [14]:
df1 = pd.read_csv('./data/combined.csv')
df2 = pd.read_csv('./data/6_02_pull.csv')

In [17]:
df2.shape

(10000, 27)

In [15]:
df = pd.concat([df1, df2])

In [16]:
df.shape

(47741, 27)

In [17]:
df.to_csv('./data/combined.csv', index=False)